In [1]:
import os,sys
smodelsPath = os.path.expanduser('~/smodels')
sys.path.append(smodelsPath)
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
from scipy.special import kn,zetac
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))
cm = plt.cm.get_cmap('RdYlBu')

In [2]:
db = Database(os.path.abspath('../../../../'))

In [3]:
exp = db.getExpResults(analysisIDs = 'CMS-EXO-13-006', dataTypes = 'efficiencyMap')[0]
print(exp)

CMS-EXO-13-006: c000,c100,c200,c300(4):THSCPM1b,THSCPM2b,THSCPM3,THSCPM4,THSCPM5,THSCPM6,THSCPM7,THSCPM8,THSCPM9(9)


### Use the trimmed database to interpolate over all original map points

In [ ]:
datasetNames = ['c000','c100','c200','c300']
txNames = ['THSCPM1b','THSCPM2b','THSCPM3','THSCPM4','THSCPM5', 'THSCPM6','THSCPM7','THSCPM8','THSCPM9']
txNames = ['THSCPM1b','THSCPM2b','THSCPM3','THSCPM4','THSCPM5', 'THSCPM6','THSCPM7','THSCPM8']
# txNames = ['THSCPM5']

resultsDict = {}
massDict = {}
for tx in txNames:
        
    resultsDict[tx] = []  
    massDict[tx] = []
    print(tx)
    for ds in datasetNames:
#         print(tx,ds)
        dataset = [dds for dds in exp.datasets if dds.dataInfo.dataId == ds][0]
        txname = [t for t in dataset.txnameList if t.txName == tx]
        if not txname:
            continue
        txname = txname[0]
        results = []
        massList = []

        data = np.genfromtxt('%s_efficiencyMaps_%s.dat' %(tx,ds),names=True)
        #For maps without the width information use only the zero width points in the original maps
#         if tx in ['THSCPM5', 'THSCPM6','THSCPM7']:
#             data = data[data['width'] == data['width'].min()]
                    
        
        for pt in data:
            massList.append([pt[mlabel] for mlabel in ['mprod','mint','mhscp'] if mlabel in data.dtype.names])
            masses = [pt[mlabel]*GeV for mlabel in ['mprod','mint','mhscp'] if mlabel in data.dtype.names]
#             if max(masses) > 3000*GeV:continue
            width = 0.0
            if 'width' in data.dtype.names:
                width = pt['width']
             
            mass = masses[:]            
            mass[-1] = tuple([mass[-1],width*GeV])
            if tx == 'THSCPM7':
                mass = [mass[:],[mass[0],mass[-1]]]
            else:
                mass = [mass]*2

            eff = pt[ds]
            effInterp = txname.txnameData.getValueFor(mass)
            #Apply reweighting for these topologies:
            if tx in ['THSCPM5', 'THSCPM6','THSCPM7']:
                widths = []
                for ibr,br in enumerate(mass):
                    widths.append([])
                    for iw,w in enumerate(br):
                        if isinstance(w,tuple):
                            widths[ibr].append(w[1])
                        else:
                            widths[ibr].append(float('inf')*GeV)
                reweightFactor = txname.txnameData.reweightF(widths)
                effInterp *= reweightFactor
                
            results.append([eff,effInterp,width])
            
        
        resultsDict[tx] += results
        massDict[tx] += massList
        

THSCPM1b
THSCPM2b
THSCPM3
THSCPM4
THSCPM5
THSCPM6
THSCPM7


### Verify convex hull (there should be no points outside the grid)

In [ ]:
for tx in resultsDict:
    resultsDict[tx] = np.array(resultsDict[tx])
    massDict[tx] = np.array(massDict[tx])
    print(tx,'# points outside the grid:',len(np.where(resultsDict[tx][:,1] == None)[0]))

### Compute relative difference for each topology

In [ ]:
relDiffDict = {}
for tx,results in resultsDict.items():
    relDiffDict[tx] = np.zeros(len(results))
    relDiffDict[tx] = np.divide(np.abs(results[:,1]-results[:,0]),results[:,0],where=(results[:,0]>0),
                                out=relDiffDict[tx])

### Plot the difference

In [ ]:
nr = len([tx for tx in relDiffDict if relDiffDict[tx].max() > 1e-4])
fig,ax = plt.subplots(nrows = int(np.ceil(nr/2.0)),ncols=2,figsize = (12,15),sharex=False)
i = 0
j = 0
for tx in relDiffDict:
    
    cond = massDict[tx][:,-1] > 0
    xpts = resultsDict[tx][cond][:,0]
    ypts = relDiffDict[tx][cond]
#     zpts = (massDict[tx][cond][:,-min(2,len(massDict[tx][0]))]-massDict[tx][cond][:,-1])
    zpts = massDict[tx][cond][:,-1]
    
    axx = ax[i,j].scatter(xpts,ypts,c=zpts,vmin=0.0,vmax=3500,cmap=cm)
    ax[i,j].set_yscale('log')
    ax[i,j].set_xscale('log')
    ax[i,j].set_ylim(1e-2,3e1)
    ax[i,j].set_xlim(1e-8,1)
#     ax[i,j].set_xticks(np.logspace(-3,3,16))
#     ax[i,j].text(1e-3,3e4,'%s : max diff = %1.1f %%' %(tx,100*(relDiffDict[tx].max())))
    ax[i,j].set_title('%s : %s' %('CMS-EXO-13-006',tx))
    ax[i,j].set_xlabel(r'$\epsilon_{Full}$')
    ax[i,j].set_ylabel(r'$|\epsilon_{Trim}-\epsilon_{Full}|/\epsilon_{Full}$')
    ax[i,j].grid()    
    j += 1
    if j >= ax.shape[1]:
        cb = fig.colorbar(axx, ax=ax[i,j-1],label=r'$m_{HSCP}$ (GeV)')
        i += 1
        j =0
        
if j == 1:
    ax[i,1].set_axis_off()
        
plt.tight_layout()

plt.savefig('databaseCheck.png')
plt.show()



In [ ]:
data = np.genfromtxt('THSCPM5_efficiencyMaps_c000.dat',names=True)
res = resultsDict['THSCPM5'][:len(data)]
diff = relDiffDict['THSCPM5'][:len(data)]

In [ ]:
condDict =  {'mprod' : 3200.0, 'mint' : 3005.0, 'mhscp' : None, 'width' : None}
condition = [True]*len(data)
condStr = ''
for var,val in condDict.items():
    if val is None: continue
    condition = condition & (data[var] == val)
    condStr += '%s = %1.2e, ' %(var,val)
condStr = condStr[:-2]

xvar = 'mhscp'
yvar = 'width'

fig,ax = plt.subplots(nrows = 1,ncols=2,figsize = (12,4),sharey=True)
resSlice = res[condition]
dataSlice  = data[condition]



ax[0].scatter(dataSlice[xvar],dataSlice[yvar],c=resSlice[:,0],
            cmap=cm,vmin=resSlice[:,0].min(),vmax=resSlice[:,0].max())
ax[0].set_yscale('log')
ax[0].set_title('Full\n (%s)' %condStr,fontsize=14)
ax[0].set_ylim(1e-22,1e7)
axx = ax[1].scatter(dataSlice[xvar],dataSlice[yvar],c=resSlice[:,1],
            cmap=cm,vmin=resSlice[:,0].min(),vmax=resSlice[:,0].max())

ax[1].set_title('Trimmed\n (%s)' %condStr,fontsize=14)
ax[1].set_ylim(1e-22,1e7)
ax[0].set_ylabel(yvar)
ax[0].set_xlabel(xvar)
ax[1].set_xlabel(xvar)


plt.tight_layout()
cb = fig.colorbar(axx, ax=ax.ravel().tolist(),label=r'$\epsilon$')
plt.show()

In [ ]:
diffSlice = diff[condition]
plt.scatter(dataSlice[xvar],dataSlice[yvar],c=diffSlice,
            cmap=cm,vmin=0,vmax=30)
plt.title('Diff\n (%s)' %condStr,fontsize=12)
plt.xlabel(xvar)
plt.ylabel(yvar)
plt.yscale('log')
plt.ylim(1e-18,1e-13)
plt.colorbar(label=r'$\Delta \epsilon/\epsilon_{Full}$')
plt.show()

In [ ]:
condDict =  {'mprod' : 3200.0, 'mint' : 3005.0, 'mhscp' : 3000.0, 'width' : None}
condition = [True]*len(data)
condStr = ''
for var,val in condDict.items():
    if val is None: continue
    condition = condition & (data[var] == val)
    condStr += '%s = %1.2e, ' %(var,val)
condStr = condStr[:-2]

xvar = 'width'

fig,ax = plt.subplots(nrows = 1,ncols=2,figsize = (12,4))
resSlice = res[condition]
dataSlice  = data[condition]

ax[0].plot(dataSlice[xvar],resSlice[:,0],label='Full')
ax[0].plot(dataSlice[xvar],resSlice[:,1],label='Trimmed')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_ylim(1e-8,1)
ax[0].set_xlim(1e-18,1e-15)
ax[0].legend()

ax[1].plot(dataSlice[xvar],(resSlice[:,1]-resSlice[:,0])/resSlice[:,0])
ax[1].set_xscale('log')
ax[1].set_xlim(1e-18,1e-15)


plt.tight_layout()
plt.show()

In [ ]:
condDict =  {'mprod' : 3200.0, 'mint' : 3005.0, 'mhscp' : 2900.0, 'width' : None}
condition = [True]*len(data)
condStr = ''
for var,val in condDict.items():
    if val is None: continue
    condition = condition & (data[var] == val)
    condStr += '%s = %1.2e, ' %(var,val)
condStr = condStr[:-2]

xvar = 'width'

fig,ax = plt.subplots(nrows = 1,ncols=2,figsize = (12,4))
resSlice = res[condition]
dataSlice  = data[condition]
ax[0].plot(dataSlice[xvar],resSlice[:,0],label='Full')
ax[0].plot(dataSlice[xvar],resSlice[:,1],label='Trimmed')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_ylim(1e-8,1)
ax[0].set_xlim(1e-18,1e-15)
ax[0].legend()

ax[1].plot(dataSlice[xvar],(resSlice[:,1]-resSlice[:,0])/resSlice[:,0])
ax[1].set_xscale('log')
ax[1].set_xlim(1e-18,1e-15)


plt.tight_layout()
plt.show()

In [ ]:
condDict =  {'mprod' : 3200.0, 'mint' : 3005.0, 'mhscp' : 2800.0, 'width' : None}
condition = [True]*len(data)
condStr = ''
for var,val in condDict.items():
    if val is None: continue
    condition = condition & (data[var] == val)
    condStr += '%s = %1.2e, ' %(var,val)
condStr = condStr[:-2]

xvar = 'width'

fig,ax = plt.subplots(nrows = 1,ncols=2,figsize = (12,4))
resSlice = res[condition]
dataSlice  = data[condition]
ax[0].plot(dataSlice[xvar],resSlice[:,0],label='Full')
ax[0].plot(dataSlice[xvar],resSlice[:,1],label='Trimmed')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_ylim(1e-8,1)
ax[0].set_xlim(1e-18,1e-15)
ax[0].legend()

ax[1].plot(dataSlice[xvar],(resSlice[:,1]-resSlice[:,0])/resSlice[:,0])
ax[1].set_xscale('log')
ax[1].set_xlim(1e-18,1e-15)


plt.tight_layout()
plt.show()